<a href="https://colab.research.google.com/github/Swarnlataaa/Video_analytics/blob/main/Violence_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless
!pip install tensorflow
!pip install keras


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [ ]:
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.load_weights('path_to_weights.h5')


In [ ]:
# Capture video from a CCTV camera
cap = cv2.VideoCapture(0)  # 0 for the default camera

# Alternatively, you can read a video file
# cap = cv2.VideoCapture('path_to_video.mp4')


In [ ]:
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame (resize, normalize, etc.)
    frame = cv2.resize(frame, (224, 224))
    frame = frame / 255.0  # Normalize

    # Make a prediction
    prediction = model.predict(np.expand_dims(frame, axis=0))

    if prediction[0][0] > 0.5:  # Adjust the threshold as needed
        cv2.putText(frame, 'Violence Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Violence Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
